<a href="https://colab.research.google.com/github/PhanDuyNhat/NHUNG_St/blob/RAS/gpio_kt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#include <stdio.h>
#include <stdint.h>
#include <softPwm.h>
#include <wiringPi.h>
#include <stdlib.h>
#include <time.h>

#define R 11
#define G 13
#define B 15
#define bt1 37
#define bt2 31
#define bt3 33



uint8_t  LED[3] = {11,13,15};
uint8_t  btn[3] = {bt1,bt2,bt3};
uint8_t color ;
float  duty1,duty2;

int n ;
int ran; 
int min = 20, max = 100;
int state=0;
int state1,state2;
time_t start_t, end_t,st,end;
float total,tt;

uint8_t  kenh[7][3] = {{100,0,0},{0,100,0},{0,0,100}
                            ,{100,100,0},{0,100,100},{100,0,100}
                            ,{100,100,100}}; 


//ham random 
int GetRandom( int min,int  max){
    return min + (int)(rand()*(max-min+1.0)/(1.0+RAND_MAX));
 
}
//ham chuyen mau 
int chuyen_mau(void){
while(state==2|(state==5)){
for ( n = 0; n<=7;n+=1) {
if (state == 0){n=10;}
softPwmWrite(LED[0], 0.8*kenh[n][0]);
softPwmWrite(LED[1], 0.8*kenh[n][1]);
softPwmWrite(LED[2], 0.8*kenh[n][2]);
delay(1000);
}
n=0;
}
}
//ham tang giam do sang 
int led_up_down(void)
{
 for(duty1=0;duty1<=1;duty1+=0.1)
    {
    printf("Tang %f\n ",duty1); 
    softPwmWrite(LED[0], duty1*kenh[1][0]);
    softPwmWrite(LED[1], duty1*kenh[1][1]);//kenh xanh 
    softPwmWrite(LED[2], duty1*kenh[1][2]);
    delay(500);
    while(duty1>0.9){
        for(duty1=1;duty1<2;duty1+=0.1){
        if (state == 0){duty1=2;}
          printf("Giam %f\n ",duty1);  
        softPwmWrite(LED[0], (2-duty1)*kenh[1][0]);
        softPwmWrite(LED[1], (2-duty1)*kenh[1][1]);
        softPwmWrite(LED[2], (2-duty1)*kenh[1][2]);
        delay(500);
       }
    duty1=0;
       if (state == 0){return 0;}
       printf("Gia tri tra ve %f\n",duty1);
    }   
    }
}



//ham chop tat warning
int led_on(void)
{ 
  for(int l=1;l<5;l++){
  if(state==5){l=8;}
  softPwmWrite(LED[0],0.8*kenh[0][0]);
  softPwmWrite(LED[1],0.8*kenh[0][1]);
  softPwmWrite(LED[2],0.8*kenh[0][2]);
  delay(500);
  softPwmWrite(LED[0],0*kenh[0][0]);
  softPwmWrite(LED[1],0*kenh[0][1]);
  softPwmWrite(LED[2],0*kenh[0][2]);
  delay(500);
  }
}

// ngat nut nhan 1
void interrupt_1(void){
if(digitalRead(bt1)==1){state=1;printf("bt1\n"); }
}

//ngat nut 2

void interrupt_2(void){
 
  if(digitalRead(bt2)==1){
  printf("bt2\n"); 
  ran = GetRandom(20,100);
  printf("%d\n",ran);
  state=2;


}

}

// nut nhan 3 
void interrupt_3(void){
  if(digitalRead(bt3)==1){
   state=5;
    printf("bt3\n");    
    time(&start_t);
    total=0;
  }
    while((digitalRead(bt3)==1)&&(total<4)){
    time(&end_t);
    total = difftime(end_t, start_t);
    printf("Thoi gian hien tai %f\n",total);
   
   if(total>=4){printf("Thoi gian thoa dieu kien %f\n",total);state=0;}
    }
}


int main(void)
{
    wiringPiSetupPhys();
    // Tạo PWM mềm cho 3 kênh led và khai báo nút nhấn
    softPwmCreate(R, 0, 100);
    softPwmCreate(G, 0, 100);
    softPwmCreate(B, 0, 100);
    pinMode(bt1, INPUT);
    pinMode(bt2, INPUT);
    pinMode(bt3, INPUT);

    // Khởi tạo interrupt nút nhấn 
    wiringPiISR(bt1, INT_EDGE_BOTH, &interrupt_1);
    wiringPiISR(bt2, INT_EDGE_BOTH, &interrupt_2);
    wiringPiISR(bt3, INT_EDGE_BOTH, &interrupt_3);

while(1){


// che do mac dinh

while(state==0 | state ==5){


  softPwmWrite(LED[0],0.6*kenh[2][0]);
  softPwmWrite(LED[1],0.6*kenh[2][1]);
  softPwmWrite(LED[2],0.6*kenh[2][2]);
 
 
}

// che do 1
while(state==1){
  
softPwmWrite(LED[0], 0.5*kenh[1][0]);
softPwmWrite(LED[1], 0.5*kenh[1][1]);
softPwmWrite(LED[2], 0.5*kenh[1][2]);

}

//che do 2 
while(state==2){
  if(ran >80) {
  
  printf("WARNING!!!!!\n ",ran);
  time(&st);
  led_on();
  time(&end);
  tt = difftime(end, st);
  printf("4s da troi qua  %f\n",tt);
  if(tt>=4){
  chuyen_mau();
  
  }
  
  }
  else  { led_up_down();}
}



}



}
